# File Transformations - Compression

In [ ]:
import pandas as pd
import time
import dask.dataframe as dd
import dask.array as dsa
import zarr
import xarray as xr
import numpy as np
import intake
from contextlib import contextmanager
import tiledb
import os
import socket
print(socket.gethostname())

In [ ]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

dask_dir = '/mnt/shared/dask'
conda_dir = '/var/lib/pworks/miniconda3'
conda_env = 'cloud-data'
print('Conda Directory:', conda_dir, '\nConda Environment:', conda_env)

cluster = SLURMCluster(project='cg-cloudmgmt',
                       cores=4, # Number of cores in the job
                       memory='16GB', # Worker memory limit will be memory/processes
                       processes=4, # Sets number of Dask workers. Threads per dask worker will be cores/processes
                       name='gcpslurmv2basic', # Name of cluster
                       queue='compute', # Partition name
                       job_cpu=4, # Set this to the number of cpus per job
                       job_mem='16GB', # Amount of memory per job
                       walltime='01:00:00',
                       log_directory=os.path.join(dask_dir, 'logs'),
                       env_extra=[
                           'source {conda_sh}; conda activate {conda_env}'.format(
                           conda_sh = os.path.join(conda_dir, 'etc/profile.d/conda.sh'),
                           conda_env= conda_env
                           )
                       ],
                       header_skip=['--mem'],
                      )

client = Client(cluster)
print('Job Script:\n',cluster.job_script())

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/var/lib/pworks/cloud-data-benchmarks.json"
token = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')

# Bucket name/public URL that contains the data you would like to convert & data set
root = 'gs://cloud-data-benchmarks/'
data = 'ETOPO1_Ice_g_gmt4'

path = root + data

## Timing Setup

In [ ]:
class DiagnosticTimer:
    def __init__(self):
        self.diagnostics = []
        
    @contextmanager
    def time(self, **kwargs):
        tic = time.time()
        yield
        toc = time.time()
        kwargs["Preprocessing Time"] = toc - tic
        kwargs
        self.diagnostics.append(kwargs)
        
    def dataframe(self):
        df = pd.DataFrame(self.diagnostics)
        return df
    
diag_timer = DiagnosticTimer()

## Tabular Data 

In [ ]:
# Names to give CSV columns. If the file does not have column names, Dask/Pandas will use your first line of data as such.
names=['lon', 'lat', 'z']

### CSV to Partitioned Parquets

In [ ]:
name_function = lambda x: f"ETOPO1_Ice_g_gmt4_{x}.parquet"

for i in ['lz4', 'zstd', 'gzip']:
    cluster.scale(40)
    client.wait_for_workers(40)
    with diag_timer.time(conversionType='csv2partparqet/' + i):
        df = dd.read_csv(path + '.csv', assume_missing=True, header=None, names=names)
        df = df.repartition(partition_size='150MB')
        dd.to_parquet(df, path + '.' + i + '.partparquet2', name_function=name_function, storage_options={'token':token},
                      compression=i)
    cluster.scale(0)

    del df

## Gridded Data

In [ ]:
intake.open_netcdf(path + '.100MB.nc', storage_options={'token':token}).to_dask().data_vars 
# Lists all data variables contained in the data set.

In [ ]:
variable = 'SLP'

internal_chunks = intake.open_netcdf(path + '.100MB.nc').to_dask()[variable].encoding['chunksizes']
coords = intake.open_netcdf(path + '.100MB.nc').to_dask()[variable].dims

### Zarr Array

In [ ]:
from numcodecs import blosc, Blosc, gzip, bz2
compressors = blosc.list_compressors()
l = 5
compressors.remove('lz4') # Zarr was already created with LZ4 compression
compressors.append('gzip')
compressors.append('bzip2')

for i in compressors:
    cluster.scale(40)
    client.wait_for_workers(40)
    if i == 'gzip':
        compressor = gzip.GZip(level=l)
    elif i == 'bzip2':
        compressor = bz2.BZ2(level=l)
    else:
        compressor = Blosc(cname=i, clevel=l)
            
    with diag_timer.time(conversionType='netcdf2zarray.' + i):
        ds = intake.open_netcdf(path + '.100MB.nc', chunks={}).to_dask()
        da = ds[variable]
        da = da.chunk(chunks=dict(zip(coords, internal_chunks)))
        da = da.data
        
        dsa.to_zarr(da, path + '.' + i + '.zarray', storage_options={'token':token, 'compressor':compressor})
    cluster.scale(0)
    del ds, da

### Zarr Group

In [ ]:
from numcodecs import blosc, Blosc, gzip, bz2
compressors = blosc.list_compressors()
l = 5
compressors.remove('lz4') # Zarr was already created with LZ4 compression
compressors.append('gzip')
compressors.append('bzip2')

for i in compressors:
    cluster.scale(40)
    client.wait_for_workers(40)
    if i == compressors[-2]:
        compressor = gzip.GZip(level=l)
    elif i == compressors[-1]:
        compressor = bz2.BZ2(level=l)
    else:
        compressor = zarr.Blosc(cname=i, clevel=l, shuffle=Blosc.SHUFFLE, blocksize=0)
            
    with diag_timer.time(conversionType='netcdf2zgroup.' + i):
        ds = intake.open_netcdf(path + '.100MB.nc').to_dask()
        da = ds[variable]
        da = da.chunk(chunks=dict(zip(coords, internal_chunks)))
        ds = da.to_dataset()
        
        ds.to_zarr(store= path + '.' + i + '.zarr2', storage_options={'token':token}, consolidated=True,
                   encoding={variable: {"compressor": compressor}})
    cluster.scale(0)
    del ds, da
    

### NetCDF to TileDB Embedded

In [ ]:
config = tiledb.Config()
config['vfs.gcs.project_id'] = 'modular-magpie-167320' # Input your project ID here
ctx = tiledb.Ctx(config)
l = 5 # Compression level input
filters = [tiledb.GzipFilter(level=l), tiledb.ZstdFilter(level=l), tiledb.LZ4Filter(level=l), tiledb.Bzip2Filter(level=l)]

for i in filters:
    uri = path + '.' + str(i).split('F')[0] + '.tldb'
    
    with diag_timer.time(conversionType='netcdf2tldb.' + str(i).split('F')[0]):
        ds = intake.open_netcdf(path + '.100MB.nc').to_dask()
        da = ds[variable]
        da = da.chunk(chunks=dict(zip(coords, internal_chunks))).data
        
############################################################################################################################
        # TileDB Custom Schema Creation
        
        filter_list = tiledb.FilterList([i])
        
        dims = []
        for n in range(len(coords)):
            dim = tiledb.Dim(name=coords[n], domain=(0, ds[variable].encoding['original_shape'][n]-1),
                             tile=internal_chunks[n], dtype=np.uint64, filters=filter_list)
            dims.append(dim)
            
        attr = [tiledb.Attr(name=variable, dtype=np.float32, filters=filter_list)]
        dom = tiledb.Domain(dims)
        
        schema = tiledb.ArraySchema(domain=dom, attrs=attr, sparse=False)
        tiledb.Array.create(uri, schema)
        tdb_array = tiledb.open(uri, "w")
############################################################################################################################
        
        da.to_tiledb(tdb_array, storage_options={"sm.compute_concurrency_level": 2, "sm.io_concurrency_level ": 2})
    
        # Consolidation is perfomed on the array for increased read speed from cloud object storage.
        config['sm.consolidation.mode'] = 'fragment_meta'
        ctx = tiledb.Ctx(config)
        tiledb.consolidate(uri, ctx=ctx)
        config['sm.consolidation.mode'] = 'fragments'
        ctx = tiledb.Ctx(config)
        tiledb.consolidate(uri, ctx=ctx)
        config['sm.consolidation.mode'] = 'array_meta'
        ctx = tiledb.Ctx(config)
        tiledb.consolidate(uri, ctx=ctx)
    
    del ds, da, uri, dims, attr, dom, schema, tdb_array

## Present Timing Results

In [ ]:
cluster.scale(0)

In [ ]:
df = diag_timer.dataframe()
df